__Author__: Bogdan Bintu

__Email__: bbintu@g.harvard.edu

__Date__:3/4/2020

In [3]:
import cPickle as pickle
import numpy as np
import os
import matplotlib.pylab as plt
import matplotlib
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['font.size']=15
matplotlib.rcParams['font.family']='Arial'

### Load data

In [34]:
folder = r'\Data'
fid = open(folder+os.sep+r'genomic-scale-with-transcription.tsv','r')
lines = np.array([ln[:-1].split('\t')for ln in fid if len(ln)>0])
zxy = np.array(lines[1:,:3][:],dtype=np.float)
zxy = zxy.reshape([-1,2082,3])/1000 #transform to um

In [5]:
Transcr = np.array(['on' in e for e in lines[1:,8]])
Transcr_WT = Transcr.reshape([-1,1041])

In [6]:
#Please see Part 3 how to calculate densities
resWT = np.load(r'Data\densityIMR90Untreated.npy')
keep = slice(None)
AD = resWT[keep,0,0,:]
BD = resWT[keep,1,0,:]
Dratio_WT = AD/BD
Dratio_WT = Dratio_WT.reshape([-1,1041])[-len(Transcr_WT):]

good_WT = (~np.isnan(zxy[:,:,0])).reshape([-1,1041])

C:\Users\Bogdan\Anaconda3\envs\py2\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in divide
  


In [21]:
tot_distsNUC_WT_ = np.array(lines[1:,-2],dtype=np.float).reshape([-1,1041])/1000
tot_distsSC_WT_ = np.array(lines[1:,-1],dtype=np.float).reshape([-1,1041])/1000
tot_distsLAM_WT_ = np.array(lines[1:,-3],dtype=np.float).reshape([-1,1041])/1000

### Nucleoli association

In [ ]:
#distances are assumed in um
tot_dists = tot_distsNUC_WT_
asocs = 1.*np.sum(tot_dists<0.25,0)/np.sum(tot_dists>-1,0)
lens = [76, 80, 66, 63, 60, 55, 53, 48, 40, 43, 44, 44, 33, 30, 31, 30, 33, 33, 33, 33, 31, 31, 51]
edges = np.cumsum([0]+lens)
from matplotlib import cm
plt.figure(figsize=(15,3))
colors = list(cm.tab20.colors)+list(cm.Dark2.colors)
for i in range(len(edges)-1):
    plt.semilogy(np.arange(edges[i],edges[i+1]),asocs[edges[i]:edges[i+1]],'.-',color = colors[i])

### Lamina and speckle

In [24]:
nuc_cutoff = 0.25
frLAM_WT = 1.*np.sum(tot_distsLAM_WT_<nuc_cutoff,0)/np.sum(good_WT,0)
frNUC_WT = 1.*np.sum(tot_distsNUC_WT_<nuc_cutoff,0)/np.sum(good_WT,0)
frSC_WT = 1.*np.sum(tot_distsSC_WT_<nuc_cutoff,0)/np.sum(good_WT,0)

dLAM_WT = np.nanmedian(tot_distsLAM_WT_,axis=0)
dSC_WT = np.nanmedian(tot_distsSC_WT_,axis=0)
dNUC_WT = np.nanmedian(tot_distsNUC_WT_,axis=0)
dRAT_WT = np.nanmedian(Dratio_WT,axis=0)

C:\Users\Bogdan\Anaconda3\envs\py2\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in less
  
C:\Users\Bogdan\Anaconda3\envs\py2\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in less
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Bogdan\Anaconda3\envs\py2\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in less
  after removing the cwd from sys.path.


### Speckle association and A/B density

In [ ]:
fig=plt.figure()
plt.semilogy(frLAM_WT,dRAT_WT,'o',color='gray',markeredgecolor='k')
plt.ylabel("A/B density ratio")
plt.xlabel("Fraction of cells lamina-associated")
#fig.savefig(r'C:\Users\Bogdan\Dropbox\ScienceWGCT_Figures\baseimages\fr_lamina_ABdensity.pdf')

### Lamina association and A/B density

In [ ]:
fig=plt.figure()
cor = np.round(np.corrcoef(np.log(frSC_WT),np.log(dRAT_WT))[0,1],2)
plt.loglog(frSC_WT,dRAT_WT,'o',color='orange',markeredgecolor='k',label='r = '+str(cor))
plt.ylabel("A/B density ratio")
plt.xlabel("Fraction of cells speckle-associated")
plt.title('Cut-off distance: '+str(int(nuc_cutoff*1000))+' nm')
plt.legend()
#fig.savefig(r'C:\Users\Bogdan\Dropbox\ScienceWGCT_Figures\baseimages\fr_lamina_ABdensity.pdf')

### Transcriptional dependent association

In [ ]:
nuc_cutoof = 0.25
Asocs_LAM_WT = tot_distsLAM_WT_<nuc_cutoof
Asocs_NUC_WT = tot_distsNUC_WT_<nuc_cutoof
Asocs_SC_WT = tot_distsSC_WT_<nuc_cutoof
dif = (1041+100)

fig=plt.figure(figsize=(12,7))
Asocs = Asocs_LAM_WT##########################################
Transcr_WT = Transcr_WT==1
aSs = 1.*np.sum(Asocs*(~Transcr_WT),axis=0)/np.sum((~Transcr_WT)*good_WT,axis=0)
aTs = 1.*np.sum(Asocs*(Transcr_WT),axis=0)/np.sum(Transcr_WT*good_WT,axis=0)
anNs = 1.*np.sum(Transcr_WT*(~Asocs),axis=0)/np.sum((~Asocs)*good_WT,axis=0)
aNs = 1.*np.sum(Transcr_WT*(Asocs),axis=0)/np.sum(Asocs*good_WT,axis=0)
plt.semilogy(np.arange(len(aTs))+dif*0,aTs/aSs,'o',color='gray',markeredgecolor='k')
plt.plot(np.arange(len(aTs))+dif*0,[np.nanmean(aTs/aSs)]*len(aTs),lw=5,color='k')

fr = aTs/aSs
fr = 100.*np.sum(fr>1)/np.sum(fr>-1)
print(fr)

Asocs = Asocs_NUC_WT##########################################
Transcr_WT = Transcr_WT==1
aSs = 1.*np.sum(Asocs*(~Transcr_WT),axis=0)/np.sum((~Transcr_WT)*good_WT,axis=0)
aTs = 1.*np.sum(Asocs*(Transcr_WT),axis=0)/np.sum(Transcr_WT*good_WT,axis=0)
anNs = 1.*np.sum(Transcr_WT*(~Asocs),axis=0)/np.sum((~Asocs)*good_WT,axis=0)
aNs = 1.*np.sum(Transcr_WT*(Asocs),axis=0)/np.sum(Asocs*good_WT,axis=0)
plt.semilogy(np.arange(len(aTs))+dif*1,aTs/aSs,'o',color='cyan',markeredgecolor='k')
plt.plot(np.arange(len(aTs))+dif*1,[np.nanmean(aTs/aSs)]*len(aTs),lw=5,color='k')

fr = aTs/aSs
fr = 100.*np.sum(fr>1)/np.sum(fr>-1)
print(fr)

Asocs = Asocs_SC_WT##########################################
Transcr_WT = Transcr_WT==1
aSs = 1.*np.sum(Asocs*(~Transcr_WT),axis=0)/np.sum((~Transcr_WT)*good_WT,axis=0)
aTs = 1.*np.sum(Asocs*(Transcr_WT),axis=0)/np.sum(Transcr_WT*good_WT,axis=0)
anNs = 1.*np.sum(Transcr_WT*(~Asocs),axis=0)/np.sum((~Asocs)*good_WT,axis=0)
aNs = 1.*np.sum(Transcr_WT*(Asocs),axis=0)/np.sum(Asocs*good_WT,axis=0)

fr = aTs/aSs
fr = 100.*np.sum(fr>1)/np.sum(fr>-1)
print(fr)
plt.semilogy(np.arange(len(aTs))+dif*2,aTs/aSs,'o',color='orange',markeredgecolor='k')
plt.plot(np.arange(len(aTs))+dif*2,[np.nanmean(aTs/aSs)]*len(aTs),lw=5,color='k')
plt.plot([0,dif*3],[1,1],'k',lw=5)
plt.ylim([0.1,3.5])
ax1 = fig.get_axes()[0]
ax1.get_yaxis().set_major_formatter(matplotlib.ticker.ScalarFormatter())
plt.yticks([0.12,0.25,0.5,1,2])
plt.ylabel('Association enrichment')
#ax1.get_yaxis().get_major_formatter().labelOnlyBase = False
#fig.savefig(r'C:\Users\Bogdan\Dropbox\2020 Chromatin Imaging Manuscript\Revision\Bogdan_Figures\base_images\CutoffNuclearBodies\250nm_WT_association_Transcription.pdf')
#plt.ylabel('')
plt.title('250nm cutoff')

### Transcriptional dependence in different nuclear environments

In [ ]:
nuc_cutoof = 0.25
Asocs_LAM_WT = tot_distsLAM_WT_<nuc_cutoof
Asocs_NUC_WT = tot_distsNUC_WT_<nuc_cutoof
Asocs_SC_WT = tot_distsSC_WT_<nuc_cutoof
dif = (1041+100)

fig=plt.figure(figsize=(15,7))#4,7
dif = (1041+100)
cols = [[0.2,0.2,0.2],'orange','yellow',[0,0.5,1],[0.5,1,1],'white']
#cols = ['orange',[0.5,0.5,0.5],[0.25,0.25,0.25],[0.75,0.75,0.75]]
good_WT = good_WT>0
Asocslist = [good_WT,Asocs_SC_WT,(~Asocs_SC_WT)&good_WT,Asocs_LAM_WT,(~Asocs_LAM_WT)&good_WT,
             (~Asocs_SC_WT)&(~Asocs_LAM_WT)&good_WT]#&at_reg,good_WT&at_reg]
#Asocslist = [Asocs_SC_WT,(~Asocs_LAM_WT)&good_WT,good_WT]
#Asocslist = [Asocs_SC_WT,good_WT]

lims = [-0.4,0.6]
no_pts = []
for iAsocs,Asocs in enumerate(Asocslist):
    Transcr =Transcr_WT==1
    DratioT = Dratio_WT.copy()
    DratioT[~(Transcr&Asocs&good_WT)]=np.nan
    dT = np.nanmedian(DratioT,0)
    DratioS = Dratio_WT.copy()
    DratioS[~((~Transcr)&Asocs&good_WT)]=np.nan
    dS = np.nanmedian(DratioS,0)
    logTS = np.log(dT/dS)/np.log(2)
    plt.plot(np.arange(1041)+dif*iAsocs,logTS,'o',color=cols[iAsocs],markeredgecolor='k')
    plt.plot(np.arange(1041)+dif*iAsocs,[np.nanmedian(logTS)]*1041,lw=5,color='r')#cols[iAsocs])
    fr = 1.*np.sum((dT/dS)>1)/np.sum((dT/dS)>0)
    no_pts.append([np.sum(logTS<lims[0]),np.sum(logTS>lims[1])])
    print(np.sum(logTS<lims[0]),np.sum(logTS>lims[1]))
    print("Number above 0:"+str(fr))
    from scipy.stats import wilcoxon as wilcoxon
    print wilcoxon(np.log(dT),np.log(dS))
#plt.title('B regions')
#plt.semilogy(np.arange(len(frLAM_am))+dif*0,1./frLAM_am*frLAM_WT,'o',color='gray',markeredgecolor='k')
#plt.semilogy(np.arange(len(frLAM_am))+dif*0,[np.nanmean(1./frLAM_am*frLAM_WT)]*1041,lw=5,color='gray')
#plt.semilogy(np.arange(len(frNUC_am))+dif*1,1./frNUC_am*frNUC_WT,'o',color='cyan',markeredgecolor='k')
#plt.semilogy(np.arange(len(frNUC_am))+dif*1,[np.nanmean(1./frNUC_am*frNUC_WT)]*1041,lw=5,color='cyan')
plt.plot([0,dif*len(Asocslist)],[0,0],':r',lw=5)
plt.ylabel('Log2 difference of \n trans A/B density ratio')
ax1 = fig.get_axes()[0]
ax1.get_yaxis().set_major_formatter(matplotlib.ticker.ScalarFormatter())
#plt.yticks([0.5,1,2])

plt.ylim(lims)
plt.xticks([dif*iAsocs+dif/2 for iAsocs in range(len(Asocslist))],
           ['All data','Speckle associated','Not speckle-associated','Lamina-associated','Not lamina-associated','Not lamina-associated\n and not speckle-associated'],rotation=10)
plt.title('Transcribing/Silent -'+str(int(nuc_cutoof*1000))+'nm cut-off')#\nNo of cut points [above,below]:'+str(no_pts))
#fig.savefig(r'C:\Users\bogdan\Dropbox\ScienceWGCT_Figures\baseimages\Fig4_AB_enrichment_transcr_SCvsallrest_v3.pdf')
#folder = r'C:\Users\Bogdan\Dropbox\2020 Chromatin Imaging Manuscript\Revision\FinalFigures\Figure 7\Subpanels'
#import os
#fig.savefig(folder+os.sep+r'Fig7DTranscriptionAsociation_LongStrip-'+str(int(nuc_cutoof*1000))+'-.pdf')

